<a href="https://colab.research.google.com/github/Hanna413/Parsing_GB/blob/Lesson4_MongoDB/MongoDB_Parsing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup as bs
import requests
from time import sleep
from pprint import pprint
from pymongo import MongoClient


In [2]:
def page_for_parsing(vacancy_name, num_page):

    url = 'https://hh.ru/search/vacancy'

    headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',
            }

    params = {'text': vacancy_name, 'page': num_page}

    return requests.get(url=url, headers=headers, params=params)

In [3]:
def clean_salary(vacancy_salary_text, min_salary=None, max_salary=None, currency_salary=None):
  list_salary = vacancy_salary_text.replace('\u202f', '').split()
  for i in range(len(list_salary) - 1):
    if list_salary[i] == 'от':
      min_salary = int(list_salary[i + 1])
    elif list_salary[i] == 'до':
      max_salary = int(list_salary[i + 1])
    elif list_salary[i] == '–':
      min_salary = int(list_salary[i - 1])
      max_salary = int(list_salary[i + 1])
  currency_salary = list_salary[-1]

  return min_salary, max_salary, currency_salary

In [4]:
vacancy_list = []
for page in range(0, 40):
  response = page_for_parsing('python', page)
  
  #print(response.url)
  soup = bs(response.content, 'html.parser')
  vacancies = soup.find_all('div',{'class': 'vacancy-serp-item__layout'})
  for vacancy in vacancies:
    vacancy_name = vacancy.find('a').text

    vacancy_salary = vacancy.find('span', {'class', 'bloko-header-section-3'})
    if vacancy_salary:
      vacancy_salary = vacancy_salary.text.replace('\u202f', '')
      min_salary, max_salary, currency_salary = clean_salary(vacancy_salary)
    else:
      vacancy_salary = 'З/П не указана'
      min_salary, max_salary, currency_salary = None, None, None

    vacancy_link = vacancy.find('a')['href']

    vacancy_dict = {
        'name': vacancy_name,
        'salary' : vacancy_salary,
        'min_salary': min_salary,
        'max_salary': max_salary,
        'currency_salary': currency_salary,
        'link' : vacancy_link,
        'source' : 'hh.ru'
    }

    vacancy_list.append(vacancy_dict)

  page += 1
  sleep(0.1)

In [5]:
len(vacancy_list)

800

In [6]:
!apt install mongodb > log


In [7]:
!service mongodb start


 * Starting database mongodb
   ...done.


In [8]:
!pip install pymongo


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
client = MongoClient()


In [10]:
db = client.vacancies_python_hh


In [11]:
collection_vacancies_python_hh = db.vacancies

In [12]:
collection_dublicates = db.dublicates

In [13]:
def check_and_add_vacancies_in_db(vacancies):
  for vacancy in vacancies:
    if not len(list(collection_vacancies_python_hh.find({'link': vacancy['link']}))):
      collection_vacancies_python_hh.insert_one(vacancy)
    else:
      collection_dublicates.insert_one(vacancy)

In [14]:
check_and_add_vacancies_in_db(vacancy_list)

In [15]:
len(list(collection_dublicates.find()))

7

In [16]:
result_db = list(collection_vacancies_python_hh.find())


In [17]:
len(result_db)

793

In [24]:
def check_salary(collection, desired_salary, currency):
  # vacancies_for_salary = []
  for vacancy in collection.find({'$and': [{'min_salary': {'$lte': desired_salary}}, {'max_salary': {'$gte': desired_salary}}, {'currency_salary': currency}]}):
    vacancies_for_salary.append(vacancy)
    pprint(vacancy)

In [25]:
check_salary(collection_vacancies_python_hh, 150000, 'руб.')

{'_id': ObjectId('63ff9cf9d5fdee8ec6246a89'),
 'currency_salary': 'руб.',
 'link': 'https://hh.ru/vacancy/70705466?from=vacancy_search_list&query=python',
 'max_salary': 220000,
 'min_salary': 60000,
 'name': 'Python-разработчик (Django, DRF)',
 'salary': '60000 – 220000 руб.',
 'source': 'hh.ru'}
{'_id': ObjectId('63ff9cf9d5fdee8ec6246a96'),
 'currency_salary': 'руб.',
 'link': 'https://hh.ru/vacancy/77107980?from=vacancy_search_list&query=python',
 'max_salary': 240000,
 'min_salary': 120000,
 'name': 'Python developer',
 'salary': '120000 – 240000 руб.',
 'source': 'hh.ru'}
{'_id': ObjectId('63ff9cf9d5fdee8ec6246aa3'),
 'currency_salary': 'руб.',
 'link': 'https://hh.ru/vacancy/77271017?from=vacancy_search_list&query=python',
 'max_salary': 150000,
 'min_salary': 100000,
 'name': 'Python Middle Developer',
 'salary': '100000 – 150000 руб.',
 'source': 'hh.ru'}
{'_id': ObjectId('63ff9cf9d5fdee8ec6246ab3'),
 'currency_salary': 'руб.',
 'link': 'https://hh.ru/vacancy/76712573?from=vaca

[{'_id': ObjectId('63ff9cf9d5fdee8ec6246a89'),
  'name': 'Python-разработчик (Django, DRF)',
  'salary': '60000 – 220000 руб.',
  'min_salary': 60000,
  'max_salary': 220000,
  'currency_salary': 'руб.',
  'link': 'https://hh.ru/vacancy/70705466?from=vacancy_search_list&query=python',
  'source': 'hh.ru'},
 {'_id': ObjectId('63ff9cf9d5fdee8ec6246a96'),
  'name': 'Python developer',
  'salary': '120000 – 240000 руб.',
  'min_salary': 120000,
  'max_salary': 240000,
  'currency_salary': 'руб.',
  'link': 'https://hh.ru/vacancy/77107980?from=vacancy_search_list&query=python',
  'source': 'hh.ru'},
 {'_id': ObjectId('63ff9cf9d5fdee8ec6246aa3'),
  'name': 'Python Middle Developer',
  'salary': '100000 – 150000 руб.',
  'min_salary': 100000,
  'max_salary': 150000,
  'currency_salary': 'руб.',
  'link': 'https://hh.ru/vacancy/77271017?from=vacancy_search_list&query=python',
  'source': 'hh.ru'},
 {'_id': ObjectId('63ff9cf9d5fdee8ec6246ab3'),
  'name': 'Разработчик бэкэнд (Python)',
  'salary